In [1]:
import os
import cv2
import re
from collections import defaultdict
from ultralytics import YOLO
import random

WARNING  Python>=3.10 is required, but Python==3.8.18 is currently installed 


In [2]:
# Function to list files in a directory
def list_files(directory):
    try:
        return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    except Exception as e:
        return str(e)


# Generate a random color for each track ID
def get_color(track_id):
    random.seed(track_id)  # Use the track ID as the seed for consistent colors
    return tuple(random.randint(0, 255) for _ in range(3))

In [3]:
# Define the directory path
name_video = "DJI_20240308110115_0005_V"
model_name = "yolo11n_DJI_0008_V_and_0010_V_2@fine-tuning"
path_mot_output_tracker = "C:/Users/dnnxl/Documents/GitHub/drone-sort/output/trackers"
path_mot_dataset = os.path.join("C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset", name_video, "valid", "images")

In [4]:
# Define the YOLO model
model = YOLO(f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{model_name}/best.pt")

In [5]:
# Get all files in the directory
files_in_directory = [os.path.join(path_mot_dataset, file) for file in list_files(path_mot_dataset)]

# Create output directory for annotated frames
output_dir = os.path.join(path_mot_output_tracker, model_name+"@"+name_video)
os.makedirs(output_dir, exist_ok=True)

# Track history for logging
track_history = []

In [6]:
files_sorted = []
for i in range(1, len(list_files(path_mot_dataset))+1):
    files_sorted.append(os.path.join(path_mot_dataset, f"frame_{i:05d}.png"))

In [7]:
# Process each frame
for file in files_in_directory:
    frame = cv2.imread(file)
    if frame is None:
        print(f"Could not read file: {file}")
        continue

    results = model.track(frame, persist=True, tracker="botsort.yaml")

    if results[0].boxes.id is None:
        continue

    match = re.search(r'frame_\d+\.png', file)
    if match:
        frame_file = match.group(0)
        frame_id = int(frame_file.split(".")[0].split("_")[1])

    # Annotate the frame with bounding boxes
    for _, result in enumerate(results):
        for box in result.boxes:
            bbox = box.xyxy[0].tolist()  # Convert from tensor to list
            track_id = box.id.item()  # Get track id
            conf = box.conf.item()  # Get confidence score
            track_history.append((frame_id, track_id, bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1], conf, -1, -1, -1))

            # Get a unique color for this track ID
            color = get_color(track_id)

            # Draw the bounding box
            start_point = (int(bbox[0]), int(bbox[1]))
            end_point = (int(bbox[2]), int(bbox[3]))
            thickness = 2
            cv2.rectangle(frame, start_point, end_point, color, thickness)

            # Put track ID and confidence score on the frame
            label = f"ID: {track_id}, Conf: {conf:.2f}"
            label_position = (int(bbox[0]), int(bbox[1]) - 10)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Save the annotated frame
    output_path = os.path.join(output_dir, os.path.basename(file))
    cv2.imwrite(output_path, frame)

c:\Users\dnnxl\anaconda3\envs\sam-pinas-dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 384x640 1 pineapple, 231.0ms
Speed: 0.0ms preprocess, 231.0ms inference, 10.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 175.4ms
Speed: 0.0ms preprocess, 175.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 177.7ms
Speed: 14.0ms preprocess, 177.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 140.4ms
Speed: 0.0ms preprocess, 140.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 153.8ms
Speed: 12.0ms preprocess, 153.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 211.3ms
Speed: 0.0ms preprocess, 211.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 178.6ms
Speed: 5.5ms preprocess, 178.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 181.5ms
Speed: 0.0ms preprocess, 181.5ms inf